In [1]:
import pandas as pd
import random
import difflib
from sklearn import datasets # sklearn comes with some toy datasets to practice
from sklearn.preprocessing import StandardScaler
from sklearn.cluster import KMeans
import matplotlib.pyplot as plt
from sklearn.metrics import silhouette_score
import spotipy
import numpy as np
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials

In [2]:
##user_song_df.drop(['type', 'uri'], axis=1, inplace=True)
##user_song_df

In [45]:
dff = pd.read_csv("out.csv")
df_features = pd.DataFrame(dff[['danceability', 'energy', 'key', 'loudness', 'mode', 'speechiness', 'acousticness', 'instrumentalness', 'liveness', 'valence', 'tempo', "duration_ms", "time_signature"]])
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.618,0.806,7,-5.308,1,0.0317,0.017900,0.776000,0.181,0.222,140.009,259849,4
1,0.706,0.863,4,-6.386,0,0.0475,0.013100,0.834000,0.119,0.309,142.007,230074,4
2,0.792,0.919,6,-9.028,0,0.0819,0.137000,0.151000,0.274,0.711,128.968,186625,4
3,0.684,0.997,8,-5.538,1,0.0579,0.002550,0.779000,0.253,0.514,142.010,299387,4
4,0.594,0.767,5,-3.706,1,0.3710,0.001520,0.000085,0.346,0.717,129.955,140538,4
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.802,0.903,11,-3.267,0,0.0460,0.001100,0.000000,0.173,0.960,128.930,192533,4
362,0.414,0.853,11,-4.198,1,0.0453,0.018100,0.001200,0.049,0.624,93.770,171693,4
363,0.815,0.605,5,-9.118,0,0.0368,0.033000,0.684000,0.114,0.795,102.642,259893,4
364,0.734,0.735,7,-6.201,1,0.0650,0.000813,0.000032,0.559,0.729,96.806,246200,4


In [46]:
df_features.describe()

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
count,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000,366.000000
mean,0.647896,0.774898,5.579235,-7.128227,0.609290,0.073021,0.122163,0.240160,0.184603,0.561647,123.841893,239834.213115,3.989071
std,0.132430,0.167543,3.710482,2.510177,0.488578,0.088097,0.182227,0.341557,0.142827,0.255846,24.204608,55496.840852,0.165159
min,0.284000,0.089800,0.000000,-17.876000,0.000000,0.023500,0.000025,0.000000,0.017600,0.043800,67.531000,119096.000000,3.000000
25%,0.565250,0.664250,2.000000,-8.286000,0.000000,0.035350,0.006513,0.000004,0.091750,0.358250,103.741500,202447.000000,4.000000
50%,0.662000,0.820000,6.000000,-6.754000,1.000000,0.045800,0.038250,0.003855,0.124000,0.596500,128.795500,238641.000000,4.000000
75%,0.741750,0.904750,9.000000,-5.559250,1.000000,0.071525,0.160750,0.582250,0.251750,0.776000,139.948750,268899.250000,4.000000
max,0.949000,0.997000,11.000000,-0.504000,1.000000,0.911000,0.951000,0.937000,0.868000,0.975000,201.936000,536067.000000,5.000000


In [47]:
scaler = StandardScaler()
scaler.fit(df_features)
features_scaled = scaler.transform(df_features)
features_scaled_df = pd.DataFrame(features_scaled, columns = df_features.columns)
#display(X.head())
print()
display(features_scaled_df)

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,-0.226059,0.185888,0.383430,0.726131,0.800784,-0.469685,-0.572942,1.570963,-0.025259,-1.329359,0.668849,0.361141,0.066263
1,0.439351,0.526565,-0.426197,0.296092,-1.248776,-0.290091,-0.599319,1.741006,-0.459945,-0.988846,0.751509,-0.176110,0.066263
2,1.089638,0.861264,0.113554,-0.757864,-1.248776,0.100925,0.081532,-0.261399,0.626770,0.584559,0.212072,-0.960092,0.066263
3,0.272998,1.327454,0.653306,0.634379,0.800784,-0.171877,-0.657293,1.579758,0.479538,-0.186488,0.751633,1.074553,0.066263
4,-0.407535,-0.047207,-0.156321,1.365207,0.800784,3.387045,-0.662953,-0.703849,1.131567,0.608043,0.252905,-1.791672,0.066263
...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,1.165252,0.765636,1.462933,1.540334,-1.248776,-0.307141,-0.665261,-0.704097,-0.081348,1.559132,0.210500,-0.853489,0.066263
362,-1.768600,0.466797,1.462933,1.168936,0.800784,-0.315097,-0.571843,-0.700579,-0.950720,0.244046,-1.244104,-1.229520,0.066263
363,1.263552,-1.015446,-0.156321,-0.793767,-1.248776,-0.411715,-0.489965,1.301239,-0.495000,0.913331,-0.877061,0.361935,0.066263
364,0.651072,-0.238464,0.383430,0.369893,0.800784,-0.091173,-0.666838,-0.704002,2.624924,0.655010,-1.118502,0.114862,0.066263


In [48]:
kmeans = KMeans(n_clusters=6, random_state=1234)
kmeans.fit(features_scaled_df)

C:\Users\41791\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1412: FutureWarning: The default value of `n_init` will change from 10 to 'auto' in 1.4. Set the value of `n_init` explicitly to suppress the warning
  super()._check_params_vs_input(X, default_n_init=10)
C:\Users\41791\anaconda3\Lib\site-packages\sklearn\cluster\_kmeans.py:1436: UserWarning: KMeans is known to have a memory leak on Windows with MKL, when there are less chunks than available threads. You can avoid it by setting the environment variable OMP_NUM_THREADS=2.
  warnings.warn(


KMeans(n_clusters=6, random_state=1234)

In [49]:
labels = kmeans.labels_
labels

array([2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 5, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0,
       2, 5, 2, 2, 2, 2, 5, 0, 0, 2, 2, 2, 2, 2, 2, 5, 0, 2, 0, 2, 2, 0,
       1, 2, 5, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 5, 0, 2, 0, 2,
       2, 2, 0, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2,
       2, 5, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 3, 2, 2, 2, 2, 2, 2, 0, 5, 2, 2, 2,
       2, 2, 2, 2, 3, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 5, 2, 2,
       0, 2, 5, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 5, 1, 5, 5,
       5, 5, 1, 1, 0, 1, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 2,
       3, 5, 1, 2, 0, 5, 1, 1, 0, 1, 0, 5, 5, 5, 1, 2, 1, 1, 1, 1, 5, 1,
       0, 5, 2, 5, 1, 5, 1, 1, 1, 1, 1, 5, 1, 1, 1, 0, 1, 1, 1, 0, 5, 1,
       1, 0, 1, 5, 1, 0, 0, 0, 1, 1, 1, 2, 5, 0, 0, 3, 5, 1, 5, 5, 1, 5,
       5, 1, 1, 0, 5, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 4,

In [50]:
clusters = kmeans.predict(features_scaled_df)
#clusters
pd.Series(clusters).value_counts().sort_index()

0     84
1    111
2    118
3      7
4      2
5     44
Name: count, dtype: int64

In [51]:
clusters

array([2, 2, 0, 2, 0, 0, 2, 2, 0, 0, 0, 5, 2, 2, 2, 0, 2, 2, 2, 2, 0, 0,
       2, 5, 2, 2, 2, 2, 5, 0, 0, 2, 2, 2, 2, 2, 2, 5, 0, 2, 0, 2, 2, 0,
       1, 2, 5, 0, 2, 2, 2, 1, 2, 2, 0, 2, 2, 2, 2, 2, 2, 5, 0, 2, 0, 2,
       2, 2, 0, 1, 2, 0, 0, 2, 2, 2, 2, 2, 2, 2, 0, 1, 2, 2, 2, 2, 2, 2,
       2, 5, 2, 2, 2, 2, 2, 0, 2, 2, 2, 2, 2, 0, 0, 2, 0, 0, 2, 2, 2, 2,
       2, 2, 2, 1, 2, 2, 2, 2, 0, 2, 3, 2, 2, 2, 2, 2, 2, 0, 5, 2, 2, 2,
       2, 2, 2, 2, 3, 0, 2, 1, 0, 2, 0, 2, 0, 2, 0, 0, 2, 2, 2, 5, 2, 2,
       0, 2, 5, 1, 2, 0, 1, 1, 0, 0, 0, 0, 0, 2, 0, 1, 1, 1, 5, 1, 5, 5,
       5, 5, 1, 1, 0, 1, 0, 0, 0, 1, 2, 1, 1, 0, 1, 1, 0, 0, 1, 0, 1, 2,
       3, 5, 1, 2, 0, 5, 1, 1, 0, 1, 0, 5, 5, 5, 1, 2, 1, 1, 1, 1, 5, 1,
       0, 5, 2, 5, 1, 5, 1, 1, 1, 1, 1, 5, 1, 1, 1, 0, 1, 1, 1, 0, 5, 1,
       1, 0, 1, 5, 1, 0, 0, 0, 1, 1, 1, 2, 5, 0, 0, 3, 5, 1, 5, 5, 1, 5,
       5, 1, 1, 0, 5, 1, 1, 1, 1, 1, 1, 1, 0, 2, 2, 1, 0, 0, 1, 1, 1, 1,
       0, 0, 1, 1, 0, 1, 1, 1, 0, 1, 0, 0, 1, 1, 4,

In [52]:
df_features["cluster"] = clusters
df_features

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature,cluster
0,0.618,0.806,7,-5.308,1,0.0317,0.017900,0.776000,0.181,0.222,140.009,259849,4,2
1,0.706,0.863,4,-6.386,0,0.0475,0.013100,0.834000,0.119,0.309,142.007,230074,4,2
2,0.792,0.919,6,-9.028,0,0.0819,0.137000,0.151000,0.274,0.711,128.968,186625,4,0
3,0.684,0.997,8,-5.538,1,0.0579,0.002550,0.779000,0.253,0.514,142.010,299387,4,2
4,0.594,0.767,5,-3.706,1,0.3710,0.001520,0.000085,0.346,0.717,129.955,140538,4,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
361,0.802,0.903,11,-3.267,0,0.0460,0.001100,0.000000,0.173,0.960,128.930,192533,4,0
362,0.414,0.853,11,-4.198,1,0.0453,0.018100,0.001200,0.049,0.624,93.770,171693,4,1
363,0.815,0.605,5,-9.118,0,0.0368,0.033000,0.684000,0.114,0.795,102.642,259893,4,2
364,0.734,0.735,7,-6.201,1,0.0650,0.000813,0.000032,0.559,0.729,96.806,246200,4,0


In [53]:
top_100 = pd.read_csv("top_100.csv")
top_100

,Unnamed: 0,Title,Artist
0,0,Cruel Summer,Taylor Swift
1,1,Paint The Town Red,Doja Cat
2,2,Is It Over Now? (Taylor's Version) [From The V...,Taylor Swift
3,3,Snooze,SZA
4,4,Standing Next To You,Jung Kook
...,...,...,...
95,95,Clean (Taylor's Version),Taylor Swift
96,96,Bleed,The Kid LAROI
97,97,But I Got A Beer In My Hand,Luke Bryan
98,98,We Don't Fight Anymore,Carly Pearce Featuring Chris Stapleton


In [54]:
import spotipy
import json
from spotipy.oauth2 import SpotifyClientCredentials
import config


#Initialize SpotiPy with user credentials
sp = spotipy.Spotify(auth_manager=SpotifyClientCredentials(client_id= config.client_id,
                                                           client_secret= config.client_secret))

In [55]:
def play_song(track_id):
    return IFrame(src="https://open.spotify.com/embed/track/"+track_id,
       width="320",
       height="80",
       frameborder="0",
       allowtransparency="true",
       allow="encrypted-media",
      )

In [56]:
from IPython.display import IFrame

In [57]:
results_for_users= sp.search (q=user_input)
track_id= results_for_users ["tracks"]["items"][0]["id"]
print(track_id)
display(play_song (track_id))

451GvHwY99NKV4zdKPRWmv


In [58]:
audio_list=[]
audio_list.append(sp.audio_features(track_id)[0])
audio_list

[{'danceability': 0.816,
  'energy': 0.375,
  'key': 7,
  'loudness': -8.053,
  'mode': 1,
  'speechiness': 0.0996,
  'acousticness': 0.607,
  'instrumentalness': 8.66e-06,
  'liveness': 0.0933,
  'valence': 0.615,
  'tempo': 114.303,
  'type': 'audio_features',
  'id': '451GvHwY99NKV4zdKPRWmv',
  'uri': 'spotify:track:451GvHwY99NKV4zdKPRWmv',
  'track_href': 'https://api.spotify.com/v1/tracks/451GvHwY99NKV4zdKPRWmv',
  'analysis_url': 'https://api.spotify.com/v1/audio-analysis/451GvHwY99NKV4zdKPRWmv',
  'duration_ms': 191907,
  'time_signature': 4}]

In [59]:
user_song_df = pd.DataFrame(audio_list)
user_song_df = user_song_df.drop(["duration_ms", "time_signature"], inplace=True, axis=1)


#- duration_ms time_signature

In [60]:
user_song_df.drop(['type', 'uri'], axis=1, inplace=True)
user_song_df

AttributeError: 'NoneType' object has no attribute 'drop'

In [61]:
user_song_df

In [62]:
user_input= input ("Give me a song: ")
user_input_lower= user_input.lower()
matches = difflib.get_close_matches(user_input, top_100['Title'], n=2)
if (user_input_lower in top_100['Title'].str.lower().values or len(matches)!=0):
    user_input_yes_no= input(f"Did you mean '{matches[0]}' bro ? Yes or No ")
    if (user_input_yes_no=="yes"):
        random_value = top_100['Title'].sample().iloc[0]  
        print(f"You should listen to '{random_value}' on the TOP 100")
        results_for_users = sp.search (q = random_value)
        track_id = results_for_users ["tracks"]["items"][0]["id"]
        display(play_song (track_id))
    elif (user_input_yes_no=="no"):
        print ("Next time :)")       
else:
    print(f"'{user_input}' is not in the list, but we recommend this song based on similar features: ")
    
    
    user_song_df = pd.DataFrame(audio_list)
    user_song_df = user_song_df.drop(["duration_ms", "time_signature"], inplace=True, axis=1)
    results_for_users= sp.search (q=user_input)
    track_id= results_for_users ["tracks"]["items"][0]["id"]
    audio_list=[]
    audio_list.append(sp.audio_features(track_id)[0])
    audio_list
    user_song_df = pd.DataFrame(audio_list) 
    
    print(user_song_df)
    
    user_song_df= user_song_df.drop(["analysis_url", "id", "track_href", "type", "uri"], axis=1)

    print(user_song_df)
      
    features_user_scaled = scaler.transform(user_song_df)
    features_user_scaled_df = pd.DataFrame(features_user_scaled, columns = user_song_df.columns)
    display(features_user_scaled_df)
    
    results_for_users = sp.search (q = audio_list)
    display(play_song (results_for_users))
    

Give me a song: banana
'banana' is not in the list, but we recommend this song based on similar features: 
   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.816   0.375    7    -8.053     1       0.0996         0.607   

   instrumentalness  liveness  valence    tempo            type  \
0          0.000009    0.0933    0.615  114.303  audio_features   

                       id                                   uri  \
0  451GvHwY99NKV4zdKPRWmv  spotify:track:451GvHwY99NKV4zdKPRWmv   

                                          track_href  \
0  https://api.spotify.com/v1/tracks/451GvHwY99NK...   

                                        analysis_url  duration_ms  \
0  https://api.spotify.com/v1/audio-analysis/451G...       191907   

   time_signature  
0               4  
   danceability  energy  key  loudness  mode  speechiness  acousticness  \
0         0.816   0.375    7    -8.053     1       0.0996         0.607   

   instrumentalness  liveness

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,1.271113,-2.390106,0.38343,-0.368914,0.800784,0.302116,2.664261,-0.704072,-0.64013,0.208821,-0.394634,-0.864785,0.066263


TypeError: can only concatenate str (not "dict") to str

,danceability,energy,key,loudness,mode,speechiness,acousticness,instrumentalness,liveness,valence,tempo,duration_ms,time_signature
0,0.816,0.375,7,-8.053,1,0.0996,0.607,0.000009,0.0933,0.615,114.303,191907,4
